# Algoritmos de Clasificación en Julia

En pocas palabras, la clasificación es la tarea de predecir una etiqueta para una observación dada. Por ejemplo: se le dan ciertas descripciones físicas de un animal, y su tarea es clasificarlas como perro o gato. Aquí, clasificaremos las flores de iris.

Como veremos más adelante, utilizaremos diferentes clasificadores y, al final de este cuaderno, los compararemos. Definiremos nuestra función de precisión ahora mismo para quitarla del camino. Utilizaremos una función de precisión simple que devuelve la relación del número de observaciones clasificadas correctamente al número total de predicciones

In [1]:
findaccuracy(predictedvals,groundtruthvals) = sum(predictedvals.==groundtruthvals)/length(groundtruthvals)

findaccuracy (generic function with 1 method)

In [2]:
#Configurando nuestro gestor de paquetes
using Pkg;

In [3]:
#Importando los paquetes necesarios
Pkg.add("GLMNet")
Pkg.add("RDatasets")
Pkg.add("MLBase")
Pkg.add("Plots")
Pkg.add("DecisionTree")
Pkg.add("Distances")
Pkg.add("NearestNeighbors")
Pkg.add("Random")
Pkg.add("LinearAlgebra")
Pkg.add("DataStructures")
Pkg.add("LIBSVM")


   Updating registry at `~/.julia/registries/General`
   Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Resolving package versions...
  Installed glmnet_jll ─ v5.0.0+0
  Installed GLMNet ───── v0.5.1
######################################################################### 100.0%-#O#- #   #                                                                    
   Updating `~/.julia/environments/v1.4/Project.toml`
  [8d5ece8b] + GLMNet v0.5.1
   Updating `~/.julia/environments/v1.4/Manifest.toml`
  [8d5ece8b] + GLMNet v0.5.1
  [78c6b45d] + glmnet_jll v5.0.0+0
  Resolving package versions...
  Installed RData ───── v0.7.1
  Installed RDatasets ─ v0.6.9
   Updating `~/.julia/environments/v1.4/Project.toml`
  [ce6b1742] + RDatasets v0.6.9
   Updating `~/.julia/environments/v1.4/Manifest.toml`
  [df47a6cb] + RData v0.7.1
  [ce6b1742] + RDatasets v0.6.9
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia

In [4]:
# Estableciendo el uso de los paquetes importados
using GLMNet
using RDatasets
using MLBase
using Plots
using DecisionTree
using Distances
using NearestNeighbors
using Random
using LinearAlgebra
using DataStructures
using LIBSVM

┌ Info: Precompiling GLMNet [8d5ece8b-de18-5317-b113-243142960cc6]
└ @ Base loading.jl:1260
┌ Info: Precompiling RDatasets [ce6b1742-4840-55fa-b093-852dadbb1d8b]
└ @ Base loading.jl:1260
┌ Info: Precompiling DecisionTree [7806a523-6efd-50cb-b5f6-3fa6f1930dbb]
└ @ Base loading.jl:1260
┌ Info: Precompiling LIBSVM [b1bec4e5-fd48-53fe-b0cb-9723c09d164b]
└ @ Base loading.jl:1260


# Capturando los datos de prueba -> Set de datos de iris

In [6]:
iris = dataset("datasets", "iris")
first(iris, 5)

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
,Float64,Float64,Float64,Float64,Categorical…
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa


In [12]:
# Separando en la variable X la Matrix del conjunto de datos separados de la variable objetivo
X = Matrix(iris[:,1:4])
#Capturando las etiquetas (variable objetivo que nos interesa clasificar)
irislabels = iris[:,5]

150-element CategoricalArray{String,1,UInt8}:
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 ⋮
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"

In [14]:
X

150×4 Array{Float64,2}:
 5.1  3.5  1.4  0.2
 4.9  3.0  1.4  0.2
 4.7  3.2  1.3  0.2
 4.6  3.1  1.5  0.2
 5.0  3.6  1.4  0.2
 5.4  3.9  1.7  0.4
 4.6  3.4  1.4  0.3
 5.0  3.4  1.5  0.2
 4.4  2.9  1.4  0.2
 4.9  3.1  1.5  0.1
 5.4  3.7  1.5  0.2
 4.8  3.4  1.6  0.2
 4.8  3.0  1.4  0.1
 ⋮              
 6.0  3.0  4.8  1.8
 6.9  3.1  5.4  2.1
 6.7  3.1  5.6  2.4
 6.9  3.1  5.1  2.3
 5.8  2.7  5.1  1.9
 6.8  3.2  5.9  2.3
 6.7  3.3  5.7  2.5
 6.7  3.0  5.2  2.3
 6.3  2.5  5.0  1.9
 6.5  3.0  5.2  2.0
 6.2  3.4  5.4  2.3
 5.9  3.0  5.1  1.8

In [15]:
#Convirtiendo en la variable categórica a variable numérica
irislabelsmap = labelmap(irislabels)
# asignando a la variable "y" la conversión
y = labelencode(irislabelsmap, irislabels)

150-element Array{Int64,1}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3

<h4> En la clasificación, a menudo queremos utilizar algunos de los datos para ajustar un modelo, y el resto de los datos para validar (comúnmente conocidos como datos de entrenamiento y pruebas). Vamos a tener estos datos listos ahora para que podamos usarlos fácilmente en el resto de este portátil. </h4>

In [16]:
function perclass_splits(y,at)
    uids = unique(y)
    keepids = []
    for ui in uids
        curids = findall(y.==ui)
        rowids = randsubseq(curids, at) 
        push!(keepids,rowids...)
    end
    return keepids
end

perclass_splits (generic function with 1 method)

In [17]:
?randsubseq

search: randsubseq randsubseq! RandomSub StratifiedRandomSub



```
randsubseq([rng=GLOBAL_RNG,] A, p) -> Vector
```

Return a vector consisting of a random subsequence of the given array `A`, where each element of `A` is included (in order) with independent probability `p`. (Complexity is linear in `p*length(A)`, so this function is efficient even if `p` is small and `A` is large.) Technically, this process is known as "Bernoulli sampling" of `A`.

# Examples

```jldoctest
julia> rng = MersenneTwister(1234);

julia> randsubseq(rng, collect(1:8), 0.3)
2-element Array{Int64,1}:
 7
 8
```


In [18]:
trainids = perclass_splits(y,0.7)
testids = setdiff(1:length(y),trainids)

48-element Array{Int64,1}:
   8
  10
  15
  20
  22
  23
  24
  28
  34
  39
  42
  43
  48
   ⋮
 112
 113
 121
 125
 127
 128
 131
 135
 136
 138
 142
 149

Necesitaremos una función más, y esa es la función que asignará clases basadas en los valores pronosticados cuando los valores predichos son continuos.

In [20]:
assign_class(predictedvalue) = argmin(abs.(predictedvalue .- [1,2,3]))

assign_class (generic function with 1 method)

# Metodo 1 : Lasso
Es un método de análisis de regresión que realiza selección de variables y regularización para mejorar la exactitud e interpretabilidad del modelo estadístico producido por este. 

In [21]:
path = glmnet(X[trainids,:], y[trainids])
cv = glmnetcv(X[trainids,:], y[trainids])

Least Squares GLMNet Cross Validation
73 models for 4 predictors in 10 folds
Best λ 0.012 (mean loss 0.048, std 0.006)

In [22]:
# Escogiendo el mejor landa para predecir con este
path = glmnet(X[trainids,:], y[trainids])
cv = glmnetcv(X[trainids,:], y[trainids])
mylambda = path.lambda[argmin(cv.meanloss)]

path = glmnet(X[trainids,:], y[trainids],lambda=[mylambda]);

In [23]:
q = X[testids,:];
predictions_lasso = GLMNet.predict(path,q)

48×1 Array{Float64,2}:
 0.9702969197275075
 0.9324056123146393
 0.8686454010978302
 0.998754667549737
 1.0743802133251603
 0.8763461987364891
 1.2083398785963966
 0.9608633601368686
 0.8798117226404489
 0.9779977173661665
 1.1102246206854227
 0.9591305981848888
 0.9741473185468368
 ⋮
 2.7322309757573473
 2.8663477100788968
 3.0098980039910845
 2.8680804720308766
 2.5815218281721837
 2.577671429352854
 2.842931179062294
 2.4557547655099077
 3.088832004620155
 2.658338191933904
 2.9292312414100348
 2.945980723723963

# Metodo 2 : Ridge
El método Ridge tiende a contraer los coeficientes de regresión al incluir el término de penalización en la función objetivo: cuanto mayor sea λ, mayor penalización y, por tanto, mayor contracción de los coeficientes.

In [24]:
# choose the best lambda to predict with.
path = glmnet(X[trainids,:], y[trainids],alpha=0);
cv = glmnetcv(X[trainids,:], y[trainids],alpha=0)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids],alpha=0,lambda=[mylambda]);
q = X[testids,:];
predictions_ridge = GLMNet.predict(path,q)
predictions_ridge = assign_class.(predictions_ridge)
findaccuracy(predictions_ridge,y[testids])

1.0

# Metodo 3 : Elastic Net
El método red elástica supera las limitaciones de la LASSO (encogimiento mínimo absoluto y operador de selección) 

In [45]:
# Usaremos la misma función pero estableceremos alfa en 0.5 (es la combinación de LASSO y RIDGE). Usaremos la misma función pero estableceremos alfa en 0.5 (es la combinación de LASSO y RIDGE).
# Elige la mejor lambda para predecir.
path = glmnet(X[trainids,:], y[trainids],alpha=0.5);
cv = glmnetcv(X[trainids,:], y[trainids],alpha=0.5)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids],alpha=0.5,lambda=[mylambda]);
q = X[testids,:];
predictions_EN = GLMNet.predict(path,q)
predictions_EN = assign_class.(predictions_EN)
findaccuracy(predictions_EN,y[testids])

1.0

# Metodo 4 : Árboles de Decisión (Decision Trees)
Dado un conjunto de datos se fabrican diagramas de construcciones lógicas, muy similares a los sistemas de predicción basados en reglas, que sirven para representar y categorizar una serie de condiciones que ocurren de forma sucesiva, para la resolución de un problema.
Elementos

### Elementos de un Árbol de decisión

Los árboles de decisión están formados por nodos, vectores de números, flechas y etiquetas.

- Cada **nodo** se puede definir como el momento en el que se ha de tomar una decisión de entre varias posibles, lo que va haciendo que a medida que aumenta el número de nodos aumente el número de posibles finales a los que puede llegar el individuo. Esto hace que un árbol con muchos nodos sea complicado de dibujar a mano y de analizar debido a la existencia de numerosos caminos que se pueden seguir.
- Los **vectores** de números serían la solución final a la que se llega en función de las diversas posibilidades que se tienen, dan las utilidades en esa solución.
- Las **flechas** son las uniones entre un nodo y otro y representan cada acción distinta.
- Las **etiquetas** se encuentran en cada nodo y cada flecha y dan nombre a cada acción.

### Conceptos

Cuando tratemos en el desarrollo de árboles utilizaremos frecuentemente estos conceptos:

- Costo. Se refiere a dos conceptos diferentes: el costo de medición para determinar el valor de una determinada propiedad (atributo) exhibida por el objeto y el costo de clasificación errónea al decidir que el objeto pertenece a la clase X cuando su clase real es Y.
- Sobreajuste (Overfitting). Se produce cuando los datos de entrenamiento son pocos o contienen incoherencias. Al tomar un espacio de hipótesis H, se dice que una hipótesis h ∈ H sobreajusta un conjunto de entrenamiento C si existe alguna hipótesis alternativa h' ∈ H tal que h clasifica mejor que h' los elementos del conjunto de entrenamiento, pero h' clasifica mejor que h el conjunto completo de posibles instancias.
- Poda (Prunning). La poda consiste en eliminar una rama de un nodo transformándolo en una hoja (terminal), asignándole la clasificación más común de los ejemplos de entrenamiento considerados en ese nodo.
- La validación cruzada. Es el proceso de construir un árbol con la mayoría de los datos y luego usar la parte restante de los datos para probar la precisión del árbol.

### Reglas

En los árboles de decisión se tiene que cumplir una serie de reglas.

1. Al comienzo del juego se da un nodo inicial que no es apuntado por ninguna flecha, es el único del juego con esta característica.
2. El resto de nodos del juego son apuntados por una única flecha.
3. De esto se deduce que hay un único camino para llegar del nodo inicial a cada uno de los nodos del juego. No hay varias formas de llegar a la misma solución final, las decisiones son excluyentes.

En los árboles de decisiones las decisiones que se eligen son lineales, a medida que vas seleccionando entre varias opciones se van cerrando otras, lo que implica normalmente que no hay marcha atrás. En general se podría decir que las normas siguen una forma condicional: Opción 1->opción 2->opción 3->Resultado Final X Estas reglas suelen ir implícitas en el conjunto de datos a raíz del cual se construye el árbol de decisión.

In [25]:
model = DecisionTreeClassifier(max_depth=2)
DecisionTree.fit!(model, X[trainids,:], y[trainids])

DecisionTreeClassifier
max_depth:                2
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  [1, 2, 3]
root:                     Decision Tree
Leaves: 3
Depth:  2

In [26]:
q = X[testids,:];
predictions_DT = DecisionTree.predict(model, q)
findaccuracy(predictions_DT,y[testids])

0.9375

In [29]:
print_tree(model, 10)

Feature 3, Threshold 2.45
L-> 1 : 37/37
R-> Feature 3, Threshold 4.75
    L-> 2 : 29/30
    R-> 3 : 32/35


In [30]:
using ScikitLearn.CrossValidation: cross_val_score
accuracy = cross_val_score(model, X[trainids,:], y[trainids], cv=3)

┌ Info: Precompiling ScikitLearn [3646fa90-6ef7-5e7e-9f22-8aca16db6324]
└ @ Base loading.jl:1260


3-element Array{Float64,1}:
 0.8857142857142857
 0.8529411764705882
 0.9696969696969697

# Metodo 5 : Bosques Aleatorios (Random Forest)
Los bosques aleatorios son una combinación de árboles predictores tal que cada árbol depende de los valores de un vector aleatorio probado independientemente y con la misma distribución para cada uno de estos. Es una modificación sustancial de bagging que construye una larga colección de árboles no correlacionados y luego los promedia.

In [31]:
model = RandomForestClassifier(n_trees=20)
DecisionTree.fit!(model, X[trainids,:], y[trainids])

RandomForestClassifier
n_trees:             20
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           -1
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             [1, 2, 3]
ensemble:            Ensemble of Decision Trees
Trees:      20
Avg Leaves: 6.15
Avg Depth:  4.25

In [32]:
q = X[testids,:];
predictions_RF = DecisionTree.predict(model, q)
findaccuracy(predictions_RF,y[testids])

0.9791666666666666

# Metodo 6 : Nearest Neighbor method
es un método de clasificación supervisada (Aprendizaje, estimación basada en un conjunto de entrenamiento y prototipos) que sirve para estimar la función de densidad  $F(x/Cj)$ de las predictoras $x$  por cada clase $C_{j}$.

Este es un método de clasificación no paramétrico, que estima el valor de la función de densidad de probabilidad o directamente la probabilidad a posteriori de que un elemento $x$ pertenezca a la clase $C_{j}$ a partir de la información proporcionada por el conjunto de prototipos. En el proceso de aprendizaje no se hace ninguna suposición acerca de la distribución de las variables predictoras.

En el reconocimiento de patrones, el algoritmo {\displaystyle k}k-nn es usado como método de clasificación de objetos (elementos) basado en un entrenamiento mediante ejemplos cercanos en el espacio de los elementos. {\displaystyle k}k-nn es un tipo de aprendizaje vago (lazy learning), donde la función se aproxima solo localmente y todo el cómputo es diferido a la clasificación.

In [33]:
Xtrain = X[trainids,:]
ytrain = y[trainids]
kdtree = KDTree(Xtrain')

KDTree{StaticArrays.SArray{Tuple{4},Float64,1,4},Euclidean,Float64}
  Number of points: 102
  Dimensions: 4
  Metric: Euclidean(0.0)
  Reordered: true

In [35]:
queries = X[testids,:]

48×4 Array{Float64,2}:
 5.0  3.4  1.5  0.2
 4.9  3.1  1.5  0.1
 5.8  4.0  1.2  0.2
 5.1  3.8  1.5  0.3
 5.1  3.7  1.5  0.4
 4.6  3.6  1.0  0.2
 5.1  3.3  1.7  0.5
 5.2  3.5  1.5  0.2
 5.5  4.2  1.4  0.2
 4.4  3.0  1.3  0.2
 4.5  2.3  1.3  0.3
 4.4  3.2  1.3  0.2
 4.6  3.2  1.4  0.2
 ⋮              
 6.4  2.7  5.3  1.9
 6.8  3.0  5.5  2.1
 6.9  3.2  5.7  2.3
 6.7  3.3  5.7  2.1
 6.2  2.8  4.8  1.8
 6.1  3.0  4.9  1.8
 7.4  2.8  6.1  1.9
 6.1  2.6  5.6  1.4
 7.7  3.0  6.1  2.3
 6.4  3.1  5.5  1.8
 6.9  3.1  5.1  2.3
 6.2  3.4  5.4  2.3

In [36]:
idxs, dists = knn(kdtree, queries', 5, true)

([[30, 37, 1, 15, 20], [26, 2, 23, 11, 19], [14, 13, 16, 9, 28], [35, 36, 5, 15, 1], [35, 15, 5, 36, 1], [7, 3, 29, 31, 5], [20, 32, 30, 24, 15], [21, 30, 1, 15, 36], [25, 13, 14, 6, 9], [8, 12, 4, 3, 34]  …  [97, 95, 98, 94, 70], [97, 95, 72, 98, 94], [85, 93, 100, 57, 91], [93, 102, 48, 85, 57], [70, 86, 88, 84, 97], [71, 57, 91, 79, 82], [70, 84, 86, 74, 81], [79, 71, 101, 87, 75], [99, 94, 75, 101, 78], [92, 78, 75, 101, 95]], [[0.09999999999999964, 0.14142135623730964, 0.17320508075688762, 0.1999999999999999, 0.22360679774997902], [0.1, 0.17320508075688784, 0.1732050807568881, 0.17320508075688815, 0.19999999999999993], [0.46904157598234253, 0.5477225575051663, 0.5567764362830022, 0.5830951894845297, 0.5916079783099616], [0.14142135623730956, 0.2449489742783178, 0.26457513110645875, 0.31622776601683783, 0.33166247903553986], [0.24494897427831772, 0.244948974278318, 0.26457513110645897, 0.2828427124746191, 0.30000000000000016], [0.45825756949558405, 0.5099019513592785, 0.50990195135

In [40]:
c = ytrain[hcat(idxs...)]
possible_labels = map(i->counter(c[:,i]),1:size(c,2))
predictions_NN = map(i->parse(Int,string(argmax(DataFrame(possible_labels[i])[1,:]))),1:size(c,2))
findaccuracy(predictions_NN,y[testids])

0.9791666666666666

# Metodo 7: Support Vector Machines
Estos métodos están propiamente relacionados con problemas de clasificación y regresión. Dado un conjunto de ejemplos de entrenamiento (de muestras) podemos etiquetar las clases y entrenar una SVM para construir un modelo que prediga la clase de una nueva muestra. Intuitivamente, una SVM es un modelo que representa a los puntos de muestra en el espacio, separando las clases a 2 espacios lo más amplios posibles mediante un hiperplano de separación definido como el vector entre los 2 puntos, de las 2 clases, más cercanos al que se llama vector soporte. Cuando las nuevas muestras se ponen en correspondencia con dicho modelo, en función de los espacios a los que pertenezcan, pueden ser clasificadas a una o la otra clase.

Más formalmente, una SVM construye un hiperplano o conjunto de hiperplanos en un espacio de dimensionalidad muy alta (o incluso infinita) que puede ser utilizado en problemas de clasificación o regresión. Una buena separación entre las clases permitirá una clasificación correcta.

**Vamos a usar el paquete `LIBSVM`**


In [41]:
Xtrain = X[trainids,:]
ytrain = y[trainids]

102-element Array{Int64,1}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3

In [42]:
model = svmtrain(Xtrain', ytrain)

LIBSVM.SVM{Int64}(SVC, LIBSVM.Kernel.RadialBasis, nothing, 4, 3, [1, 2, 3], Int32[1, 2, 3], Float64[], Int32[], LIBSVM.SupportVectors{Int64,Float64}(36, Int32[6, 15, 15], [1, 1, 1, 1, 1, 1, 2, 2, 2, 2  …  3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [4.4 4.3 … 6.5 5.9; 2.9 3.0 … 3.0 3.0; 1.4 1.1 … 5.2 5.1; 0.2 0.1 … 2.0 1.8], Int32[8, 12, 13, 16, 18, 19, 38, 39, 41, 43  …  85, 88, 89, 91, 92, 93, 96, 100, 101, 102], LIBSVM.SVMNode[LIBSVM.SVMNode(1, 4.4), LIBSVM.SVMNode(1, 4.3), LIBSVM.SVMNode(1, 5.7), LIBSVM.SVMNode(1, 5.7), LIBSVM.SVMNode(1, 4.8), LIBSVM.SVMNode(1, 5.0), LIBSVM.SVMNode(1, 7.0), LIBSVM.SVMNode(1, 6.5), LIBSVM.SVMNode(1, 5.2), LIBSVM.SVMNode(1, 6.1)  …  LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 7.2), LIBSVM.SVMNode(1, 7.9), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 6.0), LIBSVM.SVMNode(1, 5.8), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 6.5), LIBSVM.SVMNode(1, 5.9)]), 0.0, [0.5374523436089138 0.4734461406734792; 0.0 0.454632505083804; … ; -0.0 -1.0; -0.0 -1.0],

In [43]:
predictions_SVM, decision_values = svmpredict(model, X[testids,:]')
findaccuracy(predictions_SVM,y[testids])

1.0

## VAMOS A OBSERVAR TODOS LOS RESULTADOS DE LOS MÓDELOS EVALUADOS

In [46]:
overall_accuracies = zeros(7)
methods = ["lasso","ridge","EN", "DT", "RF","kNN", "SVM"]
ytest = y[testids]
overall_accuracies[1] = findaccuracy(predictions_lasso,ytest)
overall_accuracies[2] = findaccuracy(predictions_ridge,ytest)
overall_accuracies[3] = findaccuracy(predictions_EN,ytest)
overall_accuracies[4] = findaccuracy(predictions_DT,ytest)
overall_accuracies[5] = findaccuracy(predictions_RF,ytest)
overall_accuracies[6] = findaccuracy(predictions_NN,ytest)
overall_accuracies[7] = findaccuracy(predictions_SVM,ytest)
hcat(methods, overall_accuracies)

7×2 Array{Any,2}:
 "lasso"  0.0
 "ridge"  1.0
 "EN"     1.0
 "DT"     0.9375
 "RF"     0.979167
 "kNN"    0.979167
 "SVM"    1.0